In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from matplotlib import rc
rc('figure', figsize=(15, 5))
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from timeit import default_timer as timer
import pickle
import hyperopt
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
import seaborn as sns
from timeit import default_timer as timer
from numpy.random import RandomState
from hyperopt import STATUS_OK
import csv
import ast

In [2]:
sales_test = pd.read_csv(r'test.csv')

In [4]:
def create_test(test):
    test = test.loc[(test['shop_id'].isin(sales_test['shop_id']))&(test['item_id'].isin(sales_test['item_id'])),:].copy()
    test_pred = test['cnt_shop_item'].copy()
    test.drop('cnt_shop_item', axis = 1, inplace = True)
    return(test, test_pred)

In [5]:
def create_cv_sets2(months, data):
    X_train = data.loc[~data['date_block_num'].isin(months+[34]),:].drop('cnt_shop_item', axis=1)
    X_val = data.loc[data['date_block_num'].isin(months),:]
    X_val, y_val = create_test(X_val)
    X_test = data.loc[data['date_block_num'] == 34,:].drop('cnt_shop_item',axis=1)
    y_train = data.loc[~data['date_block_num'].isin(months+[34]),'cnt_shop_item']
    return(dict({'train': X_train, 'val': X_val, 'test': X_test, 'train_y': y_train, 'val_y': y_val}))

In [6]:
def create_test_data(prediction, dict_cat):
  trtv = pd.concat([dict_cat['train'], dict_cat['val']], axis = 0)
  test_ids = dict_cat['test']['item_id'].isin(trtv['item_id']) & dict_cat['test']['shop_id'].isin(trtv['shop_id'])
  prediction[~test_ids] = 0
  return(prediction)

In [9]:
def error(pred, act):
    return(np.sqrt(np.mean(np.square(np.subtract(act,pred)))))

In [7]:
with open(r"full_data_2.pkl", "rb") as input_file:
    full_data = pickle.load(input_file)

In [8]:
dict_cat = create_cv_sets2([9,21,33], full_data)

In [36]:
def objective(params):
    global ITERATION
    ITERATION +=1
    print('ITERATION: %d' %(ITERATION))
    print('params: ', params)
    start = timer()
    rf = RandomForestRegressor(n_estimators = 50, max_depth = params['max_depth'], max_features = params['max_features'], 
                               min_samples_leaf=params['min_samples_leaf'], n_jobs = -1)
    rf.fit(dict_cat['train'], dict_cat['train_y'])
    loss = error(rf.predict(dict_cat['val']), dict_cat['val_y'])
    train_time = timer()-start
    print('loss: %.5f' %(loss))
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([ITERATION, loss, params, train_time])
    return {'iteration': ITERATION, 'loss': loss, 'params': params,   
            'train_time': train_time, 'status': STATUS_OK}

In [45]:
params_space = {
    'max_depth': hp.choice('subsample', np.arange(1,11,1, dtype = int)),
    'max_features': hp.quniform('max_features', 0.05, 1, 0.05),
    'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(10, 101, 5, dtype = int))
}

In [46]:
sample(params_space)

{'max_depth': 9, 'max_features': 0.5, 'min_samples_leaf': 95}

In [47]:
out_file = 'rf_trials_pfs4.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['iteration', 'loss', 'params', 'train_time'])
of_connection.close()

In [48]:
trials = hyperopt.Trials()
global  ITERATION
ITERATION = 0
best = hyperopt.fmin(
    objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=100,
    trials=trials,
    rstate=RandomState(ITERATION)
)

ITERATION: 1
params:  {'max_depth': 4, 'max_features': 0.35000000000000003, 'min_samples_leaf': 100}
loss: 1.95021
ITERATION: 2
params:  {'max_depth': 4, 'max_features': 0.25, 'min_samples_leaf': 10}
loss: 1.98292
ITERATION: 3
params:  {'max_depth': 4, 'max_features': 0.9500000000000001, 'min_samples_leaf': 80}
loss: 1.98167
ITERATION: 4
params:  {'max_depth': 5, 'max_features': 0.1, 'min_samples_leaf': 35}
loss: 2.03247
ITERATION: 5
params:  {'max_depth': 2, 'max_features': 0.30000000000000004, 'min_samples_leaf': 55}
loss: 2.20559
ITERATION: 6
params:  {'max_depth': 2, 'max_features': 0.45, 'min_samples_leaf': 100}
loss: 2.16243
ITERATION: 7
params:  {'max_depth': 3, 'max_features': 0.45, 'min_samples_leaf': 90}
loss: 2.03427
ITERATION: 8
params:  {'max_depth': 5, 'max_features': 0.75, 'min_samples_leaf': 70}
loss: 1.89057
ITERATION: 9
params:  {'max_depth': 8, 'max_features': 0.55, 'min_samples_leaf': 10}
loss: 1.72147
ITERATION: 10
params:  {'max_depth': 3, 'max_features': 0.8, 'mi

loss: 1.97259
ITERATION: 80
params:  {'max_depth': 10, 'max_features': 0.75, 'min_samples_leaf': 35}
loss: 1.65642
ITERATION: 81
params:  {'max_depth': 8, 'max_features': 0.5, 'min_samples_leaf': 55}
loss: 1.72499
ITERATION: 82
params:  {'max_depth': 3, 'max_features': 0.55, 'min_samples_leaf': 25}
loss: 2.03429
ITERATION: 83
params:  {'max_depth': 2, 'max_features': 0.8500000000000001, 'min_samples_leaf': 60}
loss: 2.14316
ITERATION: 84
params:  {'max_depth': 6, 'max_features': 0.7000000000000001, 'min_samples_leaf': 20}
loss: 1.81626
ITERATION: 85
params:  {'max_depth': 10, 'max_features': 0.65, 'min_samples_leaf': 90}
loss: 1.66931
ITERATION: 86
params:  {'max_depth': 1, 'max_features': 0.45, 'min_samples_leaf': 80}
loss: 2.38894
ITERATION: 87
params:  {'max_depth': 10, 'max_features': 0.75, 'min_samples_leaf': 25}
loss: 1.65401
ITERATION: 88
params:  {'max_depth': 5, 'max_features': 0.4, 'min_samples_leaf': 15}
loss: 1.87141
ITERATION: 89
params:  {'max_depth': 7, 'max_features': 0

In [57]:
rf_results = pd.read_csv('rf_trials_pfs4.csv')
rf_results.sort_values('loss', ascending=True,inplace = True)
rf_results.reset_index(inplace = True)

In [58]:
rf_results.iloc[:5,:]

,index,iteration,loss,params,train_time
0,31,32,1.652485,"{'max_depth': 10, 'max_features': 0.75, 'min_s...",617.687029
1,68,69,1.653236,"{'max_depth': 10, 'max_features': 0.65, 'min_s...",532.923826
2,94,95,1.653618,"{'max_depth': 10, 'max_features': 0.65, 'min_s...",531.906359
3,58,59,1.653962,"{'max_depth': 10, 'max_features': 0.55, 'min_s...",458.894292
4,86,87,1.654012,"{'max_depth': 10, 'max_features': 0.75, 'min_s...",618.132501


In [59]:
best_params = ast.literal_eval(rf_results.loc[0, 'params'])

In [60]:
best_params

{'max_depth': 10, 'max_features': 0.75, 'min_samples_leaf': 25}

In [61]:
rf = RandomForestRegressor(n_estimators = 50, max_depth = 15, max_features = best_params['max_features'], 
                               min_samples_leaf=best_params['min_samples_leaf'], n_jobs = -1)
rf.fit(dict_cat['train'], dict_cat['train_y'])
print('rmse with max_depth = 15 is {}'.format(error(rf.predict(dict_cat['val']), dict_cat['val_y'])))

rmse with max_depth = 15 is 1.5890628824680237


In [62]:
rf = RandomForestRegressor(n_estimators = 50, max_depth = 20, max_features = best_params['max_features'], 
                               min_samples_leaf=best_params['min_samples_leaf'], n_jobs = -1)
rf.fit(dict_cat['train'], dict_cat['train_y'])
print('rmse with max_depth = 20 is {}'.format(error(rf.predict(dict_cat['val']), dict_cat['val_y'])))

rmse with max_depth = 20 is 1.5795238900710495


Prepare data for stacking

In [67]:
stack_months = np.arange(28,34)

In [68]:
stack_x = []
for i in stack_months:
  data_train_x = full_data.loc[full_data['date_block_num']<i,:].copy()
  data_test_x = full_data.loc[full_data['date_block_num']==i,:].copy()
  data_train_x.drop('cnt_shop_item', axis = 1, inplace = True)
  data_test_x.drop('cnt_shop_item', axis = 1, inplace = True)
  data_train_y = full_data.loc[full_data['date_block_num']<i,'cnt_shop_item'].copy()
  data_test_y = full_data.loc[full_data['date_block_num']==i,'cnt_shop_item'].copy()
  print('training for month', i)
  rf_stack = RandomForestRegressor(n_estimators = 50, max_depth = 20, max_features = best_params['max_features'], 
                                   min_samples_leaf=best_params['min_samples_leaf'], n_jobs = -1)
  rf_stack.fit(data_train_x, data_train_y)
  stack_x.extend(np.squeeze(rf_stack.predict(data_test_x)).tolist())

training for month 28
training for month 29
training for month 30
training for month 31
training for month 32
training for month 33


In [71]:
np.savetxt('Submission Time Series/Stacking/rf_train_level2.csv', stack_x, delimiter=',')

In [72]:
stack_x = pd.concat([dict_cat['train'], dict_cat['val']], axis=0)
stack_y = pd.concat([dict_cat['train_y'], dict_cat['val_y']], axis = 0)
rf_stack = RandomForestRegressor(n_estimators = 50, max_depth = 20, max_features = best_params['max_features'], 
                                 min_samples_leaf=best_params['min_samples_leaf'], n_jobs = -1)
rf_stack.fit(stack_x, stack_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=0.75, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=25, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [81]:
stack_level2_pred = rf_stack.predict(dict_cat['test'])

In [82]:
stack_level2_pred_y = stack_level2_pred-1
#stack_level2_pred_y = create_test_data(stack_level2_pred_y, dict_cat)
np.savetxt('Submission Time Series/Stacking/rf_test_level2.csv', stack_level2_pred_y, delimiter=',')

In [83]:
import pickle 
with open('Submission Time Series/Stacking/rf_final.pkl','wb') as handle:
    pickle.dump(rf_stack,handle,protocol=-1)